In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

from autocnet.examples import get_path
from autocnet.graph.network import CandidateGraph
from autocnet.matcher.matcher import FlannMatcher

from IPython.display import display

%pylab qt4

Populating the interactive namespace from numpy and matplotlib


## Generate a 2 image adjacenecy graph

In [2]:
#Point to the adjacency Graph
adjacency = get_path('two_image_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

#Apply SIFT to extract features
cg.extract_features(method='sift')

#Match
cg.match_features(k=5)
    
#Apply outlier detection
cg.symmetry_checks()
cg.ratio_checks()

#Compute a homography and apply RANSAC
cg.compute_homographies(clean_keys=['ratio', 'symmetry'])

## Plot features at an individual node, e.g. a single image

In [5]:
ax1 = plt.subplot(1,1,1)
ax = cg.node[0].plot(ax=ax1, color='y')

/Users/jlaura/anaconda3/envs/autocnet/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


#### The same, but a different node, with the default plot arguments

In [ ]:
cg.node[1].plot()

# Adaptive Non-Maximal Suppression

This takes a bit to run.  Do these results look correct?

In [ ]:
cg.node[0].anms()

Here is an example of how the `ax` argument can be used.  Classic matplotlib syntax where a 2 subplot plt is created and the autocnet visualization pipes to the correct axis.

In [ ]:
ax1 = plt.subplot(1,2,1)
ax2 = plt.subplot(1,2,2)

cg.node[0].plot(ax=ax1, color='r')
cg.node[0].plot(clean_keys=['anms'],ax=ax2, color='r')

## Plotting Matches on an Edge
The plotting capability on a given node is limited to a single image; one can envision the node as being the image with all associated metadata and derived information.  The edge represents the overlap between images and resultant shared information, e.g. point correspondences, a homography, etc.

#### Plot the matches between an edge using two outlier detector masks
To get a rough idea of what a 'good' results should be, we should see no, or few, lines which intersect.

In [ ]:
cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry'])

#### Now plot with the added, ransac computed mask

In [6]:
cg.edge[0][1].plot(clean_keys=['ratio', 'symmetry', 'ransac'])

/Users/jlaura/anaconda3/envs/autocnet/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


## Compute Coverage Metric
We compute a coverage metric by utilizing the homography to project the destination image corner pixel coordinates into the source image and computing the intersection.  This is a rough estimate that is as good (or poor) as the homography.

In [ ]:
#Ideal coverage would be 1.0
cg.edge[0][1].coverage_ratio(clean_keys=['ransac'])

## Viewing Keypoint Information
Here we want to explore the attributes of the keypoints, using the masking information, e.g. the outlier detection methods.  The question is, what are the characteristics of those keypoints that have made it through the outlier detection.

In [3]:
skp, dkp = cg.edge[0][1].keypoints(clean_keys=['ratio', 'symmetry', 'ransac'])
display(skp)
display(dkp)

,x,y,response,size,angle,octave,layer
36,679.339172,24.052181,0.021814,2.094834,2.652649,8,1
72,671.298035,41.698978,0.028071,2.126071,131.257462,8,1
78,605.203491,45.125435,0.015526,2.022276,314.988495,8,1
1605,243.044159,78.161858,0.018780,2.403311,202.366257,8,2
169,717.156372,94.364929,0.015108,2.058029,6.962463,8,1
183,525.033936,100.739983,0.013687,2.038394,355.072388,8,1
198,342.691895,109.109680,0.018694,1.998723,8.946075,8,1
236,817.449707,125.095490,0.033609,1.866895,168.940033,8,1
273,628.527161,139.127869,0.017962,2.009296,358.698822,8,1
320,481.941742,171.238800,0.015193,2.063699,358.773743,8,1


,x,y,response,size,angle,octave,layer
41,456.289398,23.121347,0.024122,2.085330,1.632416,8,1
104,447.569397,41.040062,0.031700,2.210840,130.800323,8,1
116,379.467133,45.548111,0.016663,2.143649,314.785431,8,1
208,24.420799,84.010201,0.018924,2.256678,206.617752,8,1
234,492.561279,93.062714,0.016501,2.216076,3.690216,8,1
253,298.090240,101.928635,0.017555,2.055187,356.201263,8,1
286,122.324875,113.260345,0.019150,2.117518,12.612762,8,1
319,598.481873,122.438530,0.035559,1.859246,165.436478,8,1
371,402.179565,139.137482,0.019570,2.018008,358.593597,8,1
440,253.525299,172.913742,0.017368,2.112939,4.900787,8,1


## Subpixel Register
We suggest only subpixel registering 'good' candidate matches as the subpixel registration process can be time consuming.

In [ ]:
cg.edge[0][1].compute_subpixel_offset(clean_keys=['ransac', 'symmetry', 'ratio'])

In [ ]:
cg.edge[0][1].plot(clean_keys=['subpixel'])

In [ ]:
subp = cg.edge[0][1]._mask_arrays['subpixel']
print(cg.edge[0][1].subpixel_offsets.iloc[subp])